# 🚁 Application RAG pour Drones Agricoles

Application Streamlit complète avec RAG (Retrieval-Augmented Generation)

**✅ Sans API externe - 100% local**

## 📦 Installation des dépendances

In [ ]:
!pip install -q streamlit sentence-transformers transformers torch numpy accelerate pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 138.1 MB/s eta 0:00:00


## 📝 Création de l'application Streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Configuration de la page
st.set_page_config(
    page_title="RAG - Drones Agricoles",
    page_icon="🚁",
    layout="wide"
)

# Données exemples sur les drones agricoles
DOCUMENT_CHUNKS = [
    """Les drones agricoles sont des véhicules aériens sans pilote utilisés dans l'agriculture de précision.
    Ils permettent de surveiller les cultures, d'analyser la santé des plantes et d'optimiser l'utilisation
    des ressources comme l'eau et les engrais. Les drones modernes sont équipés de caméras multispectrales
    et de capteurs avancés.""",

    """L'application de pesticides par drone offre plusieurs avantages : précision accrue, réduction de
    l'exposition humaine aux produits chimiques, et meilleure couverture des zones difficiles d'accès.
    Les drones peuvent pulvériser de manière ciblée, réduisant ainsi l'utilisation de pesticides de 30 à 50%.""",

    """Les capteurs multispectraux montés sur les drones capturent des images dans différentes longueurs d'onde,
    permettant d'évaluer la santé des cultures. L'indice NDVI (Normalized Difference Vegetation Index) est
    couramment utilisé pour détecter le stress hydrique et les maladies des plantes avant qu'elles ne soient
    visibles à l'œil nu.""",

    """Le coût d'un drone agricole professionnel varie entre 5 000€ et 50 000€ selon les fonctionnalités.
    Les modèles d'entrée de gamme conviennent à la surveillance basique, tandis que les modèles haut de gamme
    offrent des capacités de cartographie 3D, d'analyse thermique et de pulvérisation autonome.""",

    """L'autonomie des drones agricoles varie généralement entre 20 et 45 minutes de vol. Les batteries au
    lithium-polymère sont les plus courantes. Pour couvrir de grandes surfaces, les opérateurs utilisent
    plusieurs batteries ou des stations de recharge automatiques.""",

    """La réglementation sur l'utilisation des drones agricoles varie selon les pays. En général, un permis
    de pilotage est requis pour les opérations commerciales. Les drones doivent respecter des hauteurs de vol
    maximales (souvent 120m) et des distances minimales avec les zones habitées.""",

    """L'intelligence artificielle et le machine learning transforment l'agriculture par drone. Les algorithmes
    peuvent détecter automatiquement les mauvaises herbes, compter les plantes, estimer les rendements et
    identifier les zones nécessitant une attention particulière.""",

    """Les drones agricoles peuvent également être utilisés pour la pollinisation assistée, le comptage du
    bétail, la surveillance des clôtures et des infrastructures agricoles. Certains modèles spécialisés
    peuvent même planter des graines dans des zones difficiles d'accès."""
]

# Cache pour les modèles
@st.cache_resource
def load_embedding_model():
    """Charge le modèle d'embeddings"""
    with st.spinner("Chargement du modèle d'embeddings..."):
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model

@st.cache_resource
def load_llm_model():
    """Charge le modèle de langage"""
    with st.spinner("Chargement du modèle de langage (cela peut prendre quelques minutes)..."):
        model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
    return tokenizer, model

@st.cache_data
def generate_document_embeddings(_embedding_model):
    """Génère les embeddings des documents"""
    embeddings = _embedding_model.encode(DOCUMENT_CHUNKS)
    return embeddings

def find_similar_chunks(query_embedding, document_embeddings, k=3):
    """Trouve les k chunks les plus similaires"""
    query_norm = query_embedding / np.linalg.norm(query_embedding)
    docs_norm = document_embeddings / np.linalg.norm(document_embeddings, axis=1, keepdims=True)
    similarities = np.dot(docs_norm, query_norm)
    top_k_indices = np.argsort(similarities)[::-1][:k]
    return top_k_indices, similarities[top_k_indices]

def generate_rag_response(user_query, embedding_model, document_embeddings, tokenizer, model, k=3):
    """Génère une réponse RAG"""
    query_embedding = embedding_model.encode(user_query)
    top_k_indices, similarities = find_similar_chunks(query_embedding, document_embeddings, k=k)
    retrieved_context = [DOCUMENT_CHUNKS[i] for i in top_k_indices]
    context_string = "\n\n".join(retrieved_context)

    prompt = f"""Utilisez les informations suivantes pour répondre à la question de l'utilisateur.
Si vous ne trouvez pas la réponse dans le contexte fourni, dites simplement que vous ne savez pas.
Répondez en français de manière claire et concise.

Contexte:
{context_string}

Question de l'utilisateur: {user_query}

Réponse:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    if torch.cuda.is_available():
        inputs = inputs.to(model.device)

    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )

    response = tokenizer.decode(
        generated_tokens[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )

    return response, retrieved_context, similarities

# Interface Streamlit
def main():
    st.title("🚁 Interface RAG pour Drones Agricoles")
    st.markdown("### Posez vos questions sur les drones agricoles")

    with st.sidebar:
        st.header("ℹ️ À propos")
        st.write("Application RAG locale pour les drones agricoles")

        st.header("🔧 Configuration")
        k_chunks = st.slider("Nombre de chunks", 1, 5, 3)

        st.header("📊 Base de connaissances")
        st.info(f"**{len(DOCUMENT_CHUNKS)}** documents")

        st.header("🤖 Modèles")
        st.write("**Embeddings:** all-MiniLM-L6-v2")
        st.write("**LLM:** TinyLlama-1.1B")

    try:
        embedding_model = load_embedding_model()
        tokenizer, llm_model = load_llm_model()
        document_embeddings = generate_document_embeddings(embedding_model)
        st.success("✅ Modèles chargés avec succès!")
    except Exception as e:
        st.error(f"❌ Erreur: {str(e)}")
        return

    st.markdown("---")
    user_query = st.text_input(
        "❓ Votre question:",
        placeholder="Ex: Comment fonctionnent les capteurs multispectraux ?"
    )

    st.markdown("**💡 Exemples:**")
    col1, col2, col3 = st.columns(3)

    with col1:
        if st.button("Coût d'un drone"):
            user_query = "Quel est le coût d'un drone agricole ?"

    with col2:
        if st.button("Détection stress"):
            user_query = "Comment détecter le stress hydrique ?"

    with col3:
        if st.button("Autonomie"):
            user_query = "Quelle est l'autonomie des drones ?"

    if st.button("🔍 Obtenir une Réponse", type="primary"):
        if user_query:
            with st.spinner("🤔 Génération de la réponse..."):
                try:
                    response, retrieved_context, similarities = generate_rag_response(
                        user_query, embedding_model, document_embeddings,
                        tokenizer, llm_model, k=k_chunks
                    )

                    st.markdown("---")
                    st.subheader("💬 Réponse:")
                    st.markdown(f"**{response}**")

                    st.markdown("---")
                    st.subheader("📚 Contexte Récupéré:")

                    for i, (chunk, similarity) in enumerate(zip(retrieved_context, similarities)):
                        with st.expander(f"📄 Chunk {i+1} (Similarité: {similarity:.3f})", expanded=(i==0)):
                            st.info(chunk)
                except Exception as e:
                    st.error(f"❌ Erreur: {str(e)}")
        else:
            st.warning("⚠️ Veuillez saisir une question.")

    st.markdown("---")
    st.markdown(
        "<div style='text-align: center; color: gray;'><small>Application RAG locale - Sans API</small></div>",
        unsafe_allow_html=True
    )

if __name__ == "__main__":
    main()

Writing app.py


## 🔗 Configuration de ngrok

**⚠️ IMPORTANT:** Remplacez `VOTRE_TOKEN_ICI` par votre token ngrok

Obtenez votre token sur: https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
# ⚠️ MODIFIEZ CETTE LIGNE avec votre token ngrok
NGROK_TOKEN = "yourtokenici"

!ngrok config add-authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## 🚀 Lancement de l'application

In [ ]:
from pyngrok import ngrok
import time

print("🚀 Lancement de l'application...\n")

# Arrêter les tunnels existants
ngrok.kill()

# Créer un nouveau tunnel
public_url = ngrok.connect(8501, "http")
print(f"✅ Application accessible ici: {public_url}")
print("⏳ Attendez quelques secondes que Streamlit démarre...\n")

# Lancer Streamlit en arrière-plan
!streamlit run app.py &>/dev/null &

# Attendre le démarrage
time.sleep(5)

print("🎉 C'est prêt! Cliquez sur le lien ci-dessus.")
print("💡 Pour arrêter: Runtime > Interrupt execution")